**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**May, 2022**

Re-integration of cleaned data


----

# Re-integration of cleaned surgical data


In [1]:
from __future__ import print_function
import torch

import sys, os
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True'
sys.path.insert(1, '../results/scVI/')

In [2]:
# Seed for reproducibility
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple

# scVI imports
import scvi
from scvi.dataset import AnnDatasetFromAnnData
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)



def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '../results/images/scVI_integration'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.3.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_ser

'/opt/conda/envs/scvi-singularity/bin/python'

## Loading data

In [3]:
adata = sc.read('../results/scVI/S7_scVIintegrated_adata_latent30_cleanV4_fetalSurgical_20220523.h5ad')

In [4]:
np.unique(adata.obs['donor_souporcell'])

array(['Hrv100', 'Hrv107', 'Hrv124', 'Hrv141', 'Hrv142', 'Hrv98'],
      dtype=object)

In [5]:
adata.obs['sample']

Pla_HDBR10917730_AAACCCACATGATGCT    Pla_HDBR10917730
Pla_HDBR10917730_AAACGAACATTGACCA    Pla_HDBR10917730
Pla_HDBR10917730_AAACGCTAGAATCTAG    Pla_HDBR10917730
Pla_HDBR10917730_AAACGCTAGCTGAAGC    Pla_HDBR10917730
Pla_HDBR10917730_AAACGCTTCACCTTAT    Pla_HDBR10917730
                                           ...       
Pla_HDBR12808831_TTTGGAGTCGCCATAA    Pla_HDBR12808831
Pla_HDBR12808831_TTTGGTTGTTTGGGTT    Pla_HDBR12808831
Pla_HDBR12808831_TTTGGTTTCGTTCTAT    Pla_HDBR12808831
Pla_HDBR12808831_TTTGTTGCAAGATTGA    Pla_HDBR12808831
Pla_HDBR12808831_TTTGTTGGTGCTGATT    Pla_HDBR12808831
Name: sample, Length: 72027, dtype: category
Categories (12, object): ['Pla_HDBR10917730', 'Pla_HDBR10917732', 'Pla_HDBR10917733', 'Pla_HDBR10917734', ..., 'Pla_HDBR12808826', 'Pla_HDBR12808827', 'Pla_HDBR12808829', 'Pla_HDBR12808831']

In [6]:
scvi.data.setup_anndata(
    adata,
    layer="raw_counts",
    batch_key="donor_souporcell", #samples as a batch
    categorical_covariate_keys=['sample'] #donor as a covariate of the cells
)

INFO     Using batches from adata.obs["donor_souporcell"]                                    


Using batches from adata.obs["donor_souporcell"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.layers["raw_counts"]                                          


...

Computing library size prior per batch


INFO     Successfully registered anndata object containing 72027 cells, 32743 vars, 6        
         batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates   
         and 0 extra continuous covariates.                                                  


Successfully registered anndata object containing 72027 cells, 32743 vars, 6 batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates and 0 extra continuous covariates.


INFO     Please do not further modify adata until model is trained.                          


Please do not further modify adata until model is trained.


In [7]:
models = {}

# let's try a few values
n_latent_values = [10, 20, 30]

for n_latent_value in n_latent_values:
    print('n_latent_value', n_latent_value)
    models[n_latent_value] = scvi.model.SCVI(adata, n_latent = n_latent_value)

n_latent_value 10
n_latent_value 20
n_latent_value 30


In [8]:
latent_representations = {}

for n_latent_value in n_latent_values:
    print('training model for n_latent_value:', n_latent_value)
    models[n_latent_value].train()
    
    latent_representations[n_latent_value] = models[n_latent_value].get_latent_representation()
    
    adata.obsm["X_scVI_n_latent_" + str(n_latent_value)] = latent_representations[n_latent_value]
    
    curr_df = pd.DataFrame(adata.obsm["X_scVI_n_latent_" + str(n_latent_value)])
    curr_df.to_csv('../results/scVI/20220523_obsm_with_scVI_latent_representation_n_' + str(n_latent_value) + '_fetalSurgSubset_corrected_by_donor.csv')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 10
Epoch 111/111: 100%|██████████████████████████| 111/111 [23:07<00:00, 12.50s/it, loss=1.25e+04, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 20
Epoch 111/111: 100%|██████████████████████████| 111/111 [23:16<00:00, 12.58s/it, loss=1.23e+04, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 30
Epoch 111/111: 100%|██████████████████████████| 111/111 [23:15<00:00, 12.57s/it, loss=1.23e+04, v_num=1]


## Saving data

In [9]:
adata.write('../results/scVI/S8_scVIintegrated_fetalSurgSubset_adata_20220523.h5ad')

In [10]:
adata

AnnData object with n_obs × n_vars = 72027 × 32743
    obs: 'sample', 'donor', 'termination', 'day', 'pcw', 'percent_mito', 'n_counts', 'batch', 'n_genes', 'sample_barcode', 'souporcell_id', 'donor_deconv', 'donor_souporcell', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'is_doublet', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'scrublet_doublet', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'souporcell_MFgenotype', 'identity', 'souporcell_MFidentity'
    var: 'gene_ids', 'feature_types', 'n_cells'
    uns: 'donor_souporcell_colors', 'log1p', '_scvi'
    obsm: '_scvi_extra_categoricals', 'X_scVI_n_latent_10', 'X_scVI_n_latent_20', 'X_scVI_n_latent_30'
    layers: 'raw_counts'